In [1]:
#import plugins
import biom
import os
import pandas as pd
import qiime2 as q2
from qiime2.plugins.emperor.actions import plot, biplot
from qiime2.plugins.deicode.actions import rpca
from qiime2.plugins.diversity.actions import core_metrics_phylogenetic
from qiime2.plugins.fragment_insertion.methods import filter_features
from qiime2.plugins.diversity.visualizers import beta_group_significance
from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins.metadata.visualizers import tabulate
from qiime2.plugins.diversity.methods import alpha
from qiime2.plugins import taxa

/home/kalen/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt


In [2]:
SEQ_TYPE='shotgun'

In [3]:
#import filtered table 
tree_filtered_table = q2.Artifact.load('{SEQ_TYPE}/filtered-table.qza'.format(SEQ_TYPE=SEQ_TYPE))

#import filtered metadata
q2mf_filtered = q2.Metadata.load('{SEQ_TYPE}/metadata.tsv'.format(SEQ_TYPE=SEQ_TYPE))

#import tree
rooted_tree = q2.Artifact.load('{SEQ_TYPE}/tree.qza'.format(SEQ_TYPE=SEQ_TYPE))

In [4]:
#core diversity metrics @ different sequencing depths
for rf_depth in [1000, 10000, 100000, 500000, 1000000 ]:
    core_results = core_metrics_phylogenetic(tree_filtered_table,
                                             rooted_tree,
                                             rf_depth,
                                             q2mf_filtered)
    
    os.mkdir(('{SEQ_TYPE}/core-metrics-%i'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))
    
    for name, q2art in zip(core_results._fields, core_results):
        out_ = os.path.join(('{SEQ_TYPE}/core-metrics-%i'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth),name)
        q2art.save(out_)

/home/kalen/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/kalen/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.019430697488093233 and the largest is 17.052565577019912.
  RuntimeWarning
/home/kalen/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magni

/home/kalen/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/kalen/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.12027159003833907 and the largest is 15.061091920729872.
  RuntimeWarning
/home/kalen/miniconda3/envs/qiime2-2020.6/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnit

In [5]:
#visualize core metric qzv plots in jupyter notebooks
q2.Visualization.load('{SEQ_TYPE}/core-metrics-1000/unweighted_unifrac_emperor.qzv'.format(SEQ_TYPE=SEQ_TYPE))

<visualization: Visualization uuid: afcce4cf-1460-44ad-88d6-01322d27b913>

In [6]:
#visualize core metric qzv plots in jupyter notebooks
q2.Visualization.load('{SEQ_TYPE}/core-metrics-500000/weighted_unifrac_emperor.qzv'.format(SEQ_TYPE=SEQ_TYPE))

<visualization: Visualization uuid: 8989d027-d2b2-40bb-b9c5-d42f2e25aeab>

In [7]:
#make deicode/rpca beta diversity plot
rpcares = rpca(tree_filtered_table)

#makes a directory to put the output in
os.mkdir('{SEQ_TYPE}/rpca-res-fresh'.format(SEQ_TYPE=SEQ_TYPE))
for name, q2art in zip(rpcares._fields, rpcares):
    out_ = os.path.join('{SEQ_TYPE}/rpca-res-fresh'.format(SEQ_TYPE=SEQ_TYPE),name)
    q2art.save(out_)

In [8]:
#import biplot made by rpca
bi_plot = q2.Artifact.load('{SEQ_TYPE}/rpca-res-fresh/biplot.qza'.format(SEQ_TYPE=SEQ_TYPE))
bi_plot

<artifact: PCoAResults % Properties('biplot') uuid: 91a31be7-e447-4317-ac08-46ce7218c94f>

In [9]:
empbiplt = biplot(bi_plot,q2mf_filtered)
empbiplt

Results (name = value)
-----------------------------------------------------------------------------------------
visualization = <visualization: Visualization uuid: ab8bf175-302a-4fb8-a6cb-8c1a3985ab83>

In [10]:
empbiplt.visualization

<visualization: Visualization uuid: ab8bf175-302a-4fb8-a6cb-8c1a3985ab83>

In [12]:
#save rpca visualization
empbiplt.visualization.save('{SEQ_TYPE}/rpca-res-fresh/empbiplt.qzv'.format(SEQ_TYPE=SEQ_TYPE))

'shotgun/rpca-res-fresh/empbiplt.qzv'

In [24]:
# #beta diversity significance unweighted unifrac 10000
# UnUF_dist_matx = q2.Artifact.load('core-metrics-2500/unweighted_unifrac_distance_matrix.qza')
# UnUF_species = beta_group_significance(UnUF_dist_matx,q2mf_filtered.get_column('host_subject_id'))
# #save beta diversity significance unweighted unifrac 10000 visualization
# UnUF_species.visualization.save('core-metrics-2500/UnUF_species.qzv')
# #F stat 444.439
# #pvalue 0.001

In [13]:
UnUF_dist_matx = q2.Artifact.load(('{SEQ_TYPE}/core-metrics-%i/unweighted_unifrac_distance_matrix.qza'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))
UnUF_subject = beta_group_significance(UnUF_dist_matx,q2mf_filtered.get_column('replicate_id'))
UnUF_subject.visualization.save(('{SEQ_TYPE}/core-metrics-%i/UnUF_replicate.qzv'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))

'shotgun/core-metrics-1000000/UnUF_replicate.qzv'

In [15]:
for rf_depth in [1000, 10000, 100000, 500000, 1000000 ]:
    UnUF_dist_matx = q2.Artifact.load(('{SEQ_TYPE}/core-metrics-%i/weighted_unifrac_distance_matrix.qza'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))
    UnUF_fresh = beta_group_significance(UnUF_dist_matx,q2mf_filtered.get_column('storage_solution'))
    UnUF_fresh.visualization.save(('{SEQ_TYPE}/core-metrics-%i/UnUF_storage.qzv'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))
    
    UnUF_fresh = beta_group_significance(UnUF_dist_matx,q2mf_filtered.get_column('extraction_protocol'))
    UnUF_fresh.visualization.save(('{SEQ_TYPE}/core-metrics-%i/UnUF_extract.qzv'.format(SEQ_TYPE=SEQ_TYPE))%(rf_depth))
    #F stat 1.5829
    #pvalue 0.11

In [16]:
UnUF_dist_matx = q2.Artifact.load('shotgun/core-metrics-500000/unweighted_unifrac_distance_matrix.qza')
UnUF_fresh = beta_group_significance(UnUF_dist_matx,q2mf_filtered.get_column('extraction_protocol'))
UnUF_fresh.visualization.save('shotgun/core-metrics-500000/UnUF_extract.qzv')
#F stat 1.5829
#pvalue 0.11

'shotgun/core-metrics-500000/UnUF_extract.qzv'

In [18]:
meta = q2.Metadata.load('{SEQ_TYPE}/metadata.tsv'.format(SEQ_TYPE=SEQ_TYPE))
tax = q2.Artifact.load('{SEQ_TYPE}/taxa.qza'.format(SEQ_TYPE=SEQ_TYPE))
tax_bp = taxa.actions.barplot(tree_filtered_table, tax, meta)
tax_bp.visualization

<visualization: Visualization uuid: 4210a290-839c-42d5-aab3-2d1a05cbf8d4>